In [1]:
from slg_server.core.config import settings

settings

2025-09-21 10:05:38,373 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 10:05:38,374 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("photos")
2025-09-21 10:05:38,377 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 10:05:38,379 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tracks")
2025-09-21 10:05:38,379 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 10:05:38,381 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("points")
2025-09-21 10:05:38,382 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 10:05:38,383 INFO sqlalchemy.engine.Engine COMMIT


Settings(API_V1_STR='/api/v1', SECRET_KEY=SecretStr('**********'), ACCESS_TOKEN_EXPIRE_MINUTES=11520, FRONTEND_HOST='http://localhost:3000', ENVIRONMENT='local', BACKEND_CORS_ORIGINS=[], PROJECT_NAME='Simple Local GAllery', FIRST_SUPERUSER='admin@lysz210.it', FIRST_SUPERUSER_PASSWORD=SecretStr('**********'), GALLERY_ROOT=PosixPath('/home/lysz210/PycharmProjects/simple-local-gallery/upload'), GEMINI_API_KEY=SecretStr('**********'), all_cors_origins=['http://localhost:3000'], sqlite_dsn='sqlite:////home/lysz210/PycharmProjects/simple-local-gallery/upload/photos.db', thumbnails_root=PosixPath('/home/lysz210/PycharmProjects/simple-local-gallery/upload/.thumbnails'), gemini=<google.genai.client.Client object at 0x747fc46043b0>)

In [3]:
from pathlib import Path
from slg_server.services import photos

photo = Path('bici/Z63_6062.JPG')

thumbnail = photos.ensure_thumbnail(photo)

thumbnail

PosixPath('/home/lysz210/PycharmProjects/simple-local-gallery/upload/.thumbnails/1024/bici/Z63_6062.JPG')

In [2]:
from pydantic import BaseModel, Field

class ImageInfo(BaseModel):
    description: str
    tags: list[str] = Field(description='''lowercase and single words where appropriate, or hyphenated for phrases (e.g., lowercase and single words where appropriate, or hyphenated for phrases (e.g., use "historic-structure" for "historic structure"). Use "historic-structure" for "historic structure").''')

In [3]:
from google.genai import types
gemini = settings.gemini

system_prompt = """
You are an expert Flickr photo describer and tagger.
Your task is to analyze the provided image context and generate:
1. A compelling, concise, and engaging description suitable for a Flickr photo,
    highlighting key elements, mood, and any interesting details.
2. A list of 5-15 highly relevant and popular tags that accurately categorize the image.
    Tags should be lowercase and single words where appropriate, or hyphenated for phrases (e.g., use "historic-structure" for "historic structure").
    Avoid overly generic tags unless they are truly central (e.g., "photo" is usually too generic).

Format your output as a JSON object with two keys: "description" and "tags".
Example output:
{
    "description": "A serene forest path illuminated by dappled sunlight, creating a magical atmosphere as autumn leaves carpet the ground. Perfect for a peaceful stroll.",
    "tags": ["historic-structure", "forest", "path", "sunlight", "autumn", "leaves", "nature", "woods", "serene", "peaceful", "landscape"]
}
"""
image_types = thumbnail.read_bytes()

response = gemini.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Part.from_bytes(
            data=image_types,
            mime_type='image/jpeg'
        )
    ],
    config={
        'response_mime_type': 'application/json',
        'response_schema': ImageInfo
    }
)

response.parsed

AttributeError: 'SecretStr' object has no attribute 'strip'